<a href="https://colab.research.google.com/github/SzeHanTan/ML-Project-Customer-Churn-Prediction-and-Data-Plan-Recommendation-/blob/main/IDS_Group_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

#load dataset
df = pd.read_csv('/content/drive/MyDrive/Dataset/Teleco Customer Churn.csv')

In [4]:
df.head()

,Customer ID,Gender,Senior Citizen,Dependents,Tenure(Months),Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn
0,7590-VHVEG,Female,FALSE,FALSE,1,FALSE,NaN,DSL,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,Month-to-month,TRUE,Electronic check,29.850000,29.85000038,FALSE
1,5575-GNVDE,Male,FALSE,FALSE,34,TRUE,FALSE,DSL,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,One year,FALSE,Mailed check,-59.800000,-1889.5,FALSE
2,3668-QPYBK,Male,FALSE,FALSE,2,TRUE,FALSE,DSL,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,Month-to-month,TRUE,Mailed check,53.849998,108.1500015,TRUE
3,7795-CFOCW,Male,FALSE,FALSE,45,FALSE,NaN,DSL,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,One year,FALSE,Bank transfer (automatic),42.299999,1840.75,FALSE
4,9237-HQITU,Female,FALSE,FALSE,2,TRUE,FALSE,Fiber optic,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Month-to-month,TRUE,Electronic check,70.699997,151.6499939,TRUE


In [5]:
print(df.shape)
df.info()

(7059, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7059 entries, 0 to 7058
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Customer ID        7059 non-null   object 
 1   Gender             7059 non-null   object 
 2   Senior Citizen     7059 non-null   object 
 3   Dependents         7059 non-null   object 
 4   Tenure(Months)     7059 non-null   int64  
 5   Phone Service      7059 non-null   object 
 6   Multiple Lines     6786 non-null   object 
 7   Internet Service   7059 non-null   object 
 8   Online Security    6406 non-null   object 
 9   Online Backup      6406 non-null   object 
 10  Device Protection  6406 non-null   object 
 11  Tech Support       6406 non-null   object 
 12  Streaming TV       6406 non-null   object 
 13  Streaming Movies   6406 non-null   object 
 14  Contract           7059 non-null   object 
 15  Paperless Billing  7059 non-null   object 
 16  Payment Metho

In [6]:
df.describe()

,Tenure(Months),Monthly Charges
count,7059.000000,7059.000000
mean,32.362658,64.635402
std,24.559853,30.288076
min,0.000000,-95.500000
25%,9.000000,35.400001
50%,29.000000,70.300003
75%,55.000000,89.849998
max,72.000000,118.750000


In [7]:
#Check the missing value
missing_values = df.isnull().sum()
print(missing_values)

Customer ID            0
Gender                 0
Senior Citizen         0
Dependents             0
Tenure(Months)         0
Phone Service          0
Multiple Lines       273
Internet Service       0
Online Security      653
Online Backup        653
Device Protection    653
Tech Support         653
Streaming TV         653
Streaming Movies     653
Contract               0
Paperless Billing      0
Payment Method         0
Monthly Charges        0
Total Charges          5
Churn                  1
dtype: int64


In [9]:
import numpy as np

# Assuming df is your DataFrame
# Step 1: Filter rows where 'Dependents' is 'Yes'
dependent_true = df[df['Dependents'] == 'Yes']

# Step 2: Count 'Yes' and 'No' in 'Multiple Lines' where 'Dependents' is 'Yes'
multiple_lines_counts = dependent_true['Multiple Lines'].value_counts(normalize=True)

# Print the value counts
print(multiple_lines_counts)


Multiple Lines
No                  0.507185
Yes                 0.393914
No phone service    0.098901
Name: proportion, dtype: float64


In [13]:
# Step 1: Standardize the values in 'Dependents'
df['Dependents'] = df['Dependents'].replace({'FALSE': 'No', 'TRUE': 'Yes'})

# Step 2: Standardize the values in 'Multiple Lines'
df['Multiple Lines'] = df['Multiple Lines'].replace({'FALSE': 'No', 'TRUE': 'Yes'})

# Step 3: Fill missing values in 'Multiple Lines' based on the condition
def fill_multiple_lines(row):
    if pd.isnull(row['Multiple Lines']):
        if row['Dependents'] == 'Yes':
            return 'No'
        elif row['Dependents'] == 'No':
            return 'Yes'
    return row['Multiple Lines']

df['Multiple Lines'] = df.apply(fill_multiple_lines, axis=1)

# Step 4: Verify the results
print("Unique values in 'Dependents':", df['Dependents'].unique())
print("Unique values in 'Multiple Lines':", df['Multiple Lines'].unique())
print("Missing values in 'Multiple Lines':", df['Multiple Lines'].isnull().sum())


Unique values in 'Dependents': ['No' 'Yes']
Unique values in 'Multiple Lines': ['Yes' 'No' 'No phone service']
Missing values in 'Multiple Lines': 0


In [14]:
# Features to be filled
features_to_fill = [
    'Online Security', 'Online Backup', 'Device Protection',
    'Tech Support', 'Streaming TV', 'Streaming Movies'
]

# Fill missing values for the features based on 'Internet Service'
for feature in features_to_fill:
    df.loc[(df['Internet Service'] == 'No') & (df[feature].isnull()), feature] = 'No'

# Verify the changes
missing_values_after = df.isnull().sum()
print("Missing values after filling based on 'Internet Service':")
print(missing_values_after)


Missing values after filling based on 'Internet Service':
Customer ID          0
Gender               0
Senior Citizen       0
Dependents           0
Tenure(Months)       0
Phone Service        0
Multiple Lines       0
Internet Service     0
Online Security      0
Online Backup        0
Device Protection    0
Tech Support         0
Streaming TV         0
Streaming Movies     0
Contract             0
Paperless Billing    0
Payment Method       0
Monthly Charges      0
Total Charges        5
Churn                1
dtype: int64
